pwd
source ./setup-exports.sh

In [ ]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# Load environment variables
from dotenv import load_dotenv

MONGO_URI = os.environ.get("MONGO_URI")
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")

DB_NAME = os.environ.get("DB_NAME")
COLLECTION_NAME = os.environ.get("COLLECTION_NAME")
ATLAS_VECTOR_SEARCH_INDEX_NAME = "default"
EMBEDDING_FIELD_NAME = "embedding"
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
MONGODB_COLLECTION = db[COLLECTION_NAME]

if __name__ == "__main__":
    # Load docs
    loader = PyPDFLoader("https://arxiv.org/pdf/2303.08774.pdf")
    data = loader.load()

    # Split docs
    print("Splitting documents")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    docs = text_splitter.split_documents(data)

    # Insert the documents in MongoDB Atlas Vector Search
    print("Inserting documents in MongoDB Atlas Vector Search")
    _ = MongoDBAtlasVectorSearch.from_documents(
        documents=docs,
        embedding=CohereEmbeddings(cohere_api_key=COHERE_API_KEY),
        collection=MONGODB_COLLECTION,
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    )


In [ ]:
from langserve.client import RemoteRunnable

rag_app_pinecone = RemoteRunnable("http://0.0.0.0:8000/endpoint")
rag_app_pinecone.invoke("How does agent memory work?")